In [1]:
import pandas as pd
import re
import unicodedata
import sys

def normalizar(texto):
    """Normaliza el texto a minúsculas, elimina acentos y caracteres especiales, excepto guiones."""
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto) if unicodedata.category(c) != 'Mn')
    return texto

def levenshtein_distance(s1, s2):
    """Calcula la distancia de Levenshtein entre dos cadenas."""
    if len(s1) > len(s2):
        s1, s2 = s2, s1
    
    distancias = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        nueva_distancia = [i2 + 1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                nueva_distancia.append(distancias[i1])
            else:
                nueva_distancia.append(1 + min((nueva_distancia[-1], distancias[i1], distancias[i1+1])))
        distancias = nueva_distancia
    return distancias[-1]

def corregir_sentencia(sentencia, diccionario):
    """Corrige errores de escritura en una sentencia usando la distancia de Levenshtein."""
    palabras_originales = re.findall(r'\b[\w-]+\b', sentencia)
    palabras_corregidas = []
    
    for palabra in palabras_originales:
        palabra_normalizada = normalizar(palabra)
        mejor_coincidencia = palabra
        min_distancia = float('inf')
        
        # Si la palabra ya está en el diccionario, no se corrige
        if palabra_normalizada in diccionario:
            palabras_corregidas.append(palabra)
            continue
            
        # Busca la palabra más cercana en el diccionario
        for palabra_dic in diccionario:
            dist = levenshtein_distance(palabra_normalizada, palabra_dic)
            if dist < min_distancia:
                min_distancia = dist
                mejor_coincidencia = palabra_dic
        
        # Corrige solo si la distancia es suficientemente pequeña (e.g., 1 o 2)
        if min_distancia <= 2:
            palabras_corregidas.append(mejor_coincidencia)
        else:
            palabras_corregidas.append(palabra)
            
    return ' '.join(palabras_corregidas)

if __name__ == "__main__":
    
    # 1. Cargar el diccionario desde un archivo de Excel
    diccionario_file = "diccionario_data.xlsx"
    try:
        df_diccionario = pd.read_excel(diccionario_file)
        # Convertir la columna de palabras a un conjunto para una búsqueda rápida
        diccionario = set(df_diccionario['Palabras en el Diccionario'].dropna())
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo del diccionario: {diccionario_file}")
        print("Asegúrate de haber generado el diccionario con el script anterior.")
        sys.exit(1)

    print("Diccionario cargado exitosamente.")

    # 2. Pedir al usuario que ingrese una sentencia
    sentencia_a_corregir = input("Por favor, ingresa una sentencia para corregir: ")

    # 3. Corregir la sentencia
    sentencia_corregida = corregir_sentencia(sentencia_a_corregir, diccionario)

    # 4. Imprimir los resultados
    print("\n--- Resultados de la corrección ---")
    print(f"Sentencia original: {sentencia_a_corregir}")
    print(f"Sentencia corregida: {sentencia_corregida}")

Diccionario cargado exitosamente.


Por favor, ingresa una sentencia para corregir:  conchade pag ahora mismo sino te mayo



--- Resultados de la corrección ---
Sentencia original: conchade pag ahora mismo sino te mayo
Sentencia corregida: concha pago ahora mismo sino te mato
